In [ ]:
import pandas as pd
import os
import random
from faker import Faker
from os.path import join as path_join
from datetime import datetime as dt

Faker.seed(16042002)
faker_br = Faker('pt_BR')
faker_en = Faker('en_IN')
faker_jp = Faker('ja_JP')


CHUNKSIZE = 50000

In [ ]:
LIVRO_PATH = "livros/treated/"
ARTIGO_PATH = "artigos/treated/"
DVD_PATH = "dvds/treated/"
REVISTA_PATH = "revistas/treated/"

## Importar para o Postgres

1. Bibliotecas (OK)
2. Usuarios (OK)
3. Autores (OK)
4. Livros (OK)
5. Revistas (OK)
6. DVD's (OK)
7. Artigos (OK)
8. Autorias (OK)
9. Midia (OK)
10. Emprestimos 
11. Penalização 

In [ ]:
import psycopg2
from psycopg2.extras import execute_values

# Conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="onixlibrary",
    user="super_user",
    password="carimboatrasado",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Função para inserir dados no banco
def insert_data(table_name, data, columns):
    query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
    execute_values(cursor, query, data)
    conn.commit()

def generate_user():
    user = {}
    user['name'] = faker_br.name()
    user['email'] = faker_br.email()
    user['address'] = ' - '.join(faker_br.address().split('\n'))
    user['phone_number'] = faker_br.phone_number()
    return user

def generate_user_list():
    user = []
    user.append(faker_br.name())
    user.append(faker_br.email())
    user.append(' - '.join(faker_br.address().split('\n')))
    user.append(faker_br.phone_number())
    return user



In [ ]:

insert_data("usuario", 
            [generate_user_list() for _ in range(113_000)], 
            ['nome', 'email', 'endereco', 'telefone'])

df = pd.read_sql("SELECT * FROM usuario", conn)

df.head(10)

In [ ]:

# Crie uma lista de bibliotecas fictícias
fake_libraries = [
    "Biblioteca Onix Jaçanã",
    "Biblioteca Onix Tatuapé",
    "Biblioteca Onix Vila Prudente",
    "Biblioteca Onix Paulista",
    "Biblioteca Onix Santana",
    "Biblioteca Onix Santo André",
    "Biblioteca Onix São Bernardo do Campo",
    "Biblioteca Onix São Caetano do Sul",
    "Biblioteca Onix São José dos Campos",
    "Biblioteca Onix Higienópolis",
    "Biblioteca Onix Vila Madalena",
]
fake_library_addresses = [
    "Rua das Palmeiras, 123 – Jaçanã, São Paulo – SP, 02260-000",
    "Av. Álvaro Ramos, 456 – Tatuapé, São Paulo – SP, 03310-000",
    "Rua José Zappi, 789 – Vila Prudente, São Paulo – SP, 03138-000",
    "Av. Paulista, 1001 – Bela Vista, São Paulo – SP, 01311-100",
    "Rua Voluntários da Pátria, 321 – Santana, São Paulo – SP, 02010-000",
    "Rua General Glicério, 159 – Centro, Santo André – SP, 09015-330",
    "Av. Faria Lima, 987 – Centro, São Bernardo do Campo – SP, 09710-000",
    "Rua Alegre, 202 – Santa Paula, São Caetano do Sul – SP, 09560-300",
    "Av. Adhemar de Barros, 1550 – Jardim São Dimas, São José dos Campos – SP, 12245-010",
    "Rua Itacolomi, 415 – Higienópolis, São Paulo – SP, 01239-000",
    "Rua Harmonia, 678 – Vila Madalena, São Paulo – SP, 05435-001",
]
insert_data("biblioteca", 
            [[bib, add] for bib, add in zip(fake_libraries, fake_library_addresses)], 
            ['nome', 'endereco'])


In [ ]:

df = pd.read_csv("livros/authors.csv", dtype=str, sep=",")
max_id = df['author_id'].astype(int).max() + 1

authors_set = df['author_name'].tolist()
id_authors_set = df['author_id'].tolist()


df = pd.concat([pd.read_csv(path_join(ARTIGO_PATH, file), dtype=str, sep=",") for file in os.listdir(ARTIGO_PATH)], ignore_index=True)[['autores']]
df = pd.concat([df, pd.concat(
    [pd.read_csv(path_join(REVISTA_PATH, file), dtype=str, sep=",") for file in os.listdir(REVISTA_PATH)], ignore_index=True)[['autores']]
], ignore_index=True)


authors = set()
for idx, row in df.iterrows():
    authors.update({ a.strip() for a in str(row['autores']).split('|')})

authors = { str(k + max_id): v for k, v in enumerate(authors) if v.strip()}
id_authors = list(authors.keys())
names_authors = list(authors.values())

id_authors.extend(id_authors_set)
names_authors.extend(authors_set)

# Crie uma lista de autores fictícios


birth = [faker_br.date_between(start_date='-100y', end_date='-18y').strftime('%Y-%m-%d') for _ in range(len(names_authors))]
death = []
for i in range(len(names_authors)):
    if i % 20 == 0:
        death.append(faker_br.date_between(start_date='-100y', end_date='-18y').strftime('%Y-%m-%d'))
    else:
        death.append(None)


data = [
    (id, name, dt.strptime(b, "%Y-%m-%d").date(), dt.strptime(d, "%Y-%m-%d").date() if d else d)
    for id, name, b, d in zip(id_authors, names_authors, birth, death)
]


print("Inserindo")
insert_data("Autores", data, ['id_autor', 'nome', 'data_nascimento', 'data_falecimento'])

In [ ]:
biblio_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

author_df = pd.read_sql("SELECT * FROM autores", conn)

author_df.head(10)

In [ ]:

livros = pd.concat([pd.read_csv(path_join(LIVRO_PATH, file), dtype=str, sep=",") for file in os.listdir(LIVRO_PATH)], ignore_index=True)
leng = livros.shape[0]
for idx, row in livros.iterrows():
    try:
        biblios = random.randint(1, 5)
        if idx % 10000 == 0: print(f"Livros: {idx}/{leng}")
        for id_biblioteca in random.sample(biblio_ids, biblios):
            cursor.execute("""
                INSERT INTO Midia (tipo_midia, condicao, id_biblioteca)
                VALUES (%s, %s, %s)
                RETURNING id_midia;
            """, ('livro', random.choice(['NOVO', 'SEMINOVO', 'DESGASTADO', 'DANIFICADO', 'ARQUIVADO']), id_biblioteca))

            id_midia = cursor.fetchone()[0]

            # 2. Inserir em Livros (com o mesmo id)
            cursor.execute("""
                INSERT INTO Livros (id_livro, titulo, ISBN, numero_paginas, editora, data_publicacao)
                VALUES (%s, %s, %s, %s, %s, %s);
            """, (id_midia, row['title'], str(row['isbn13']).replace('-', ''), row['pages'] if int(row['pages']) < 10000 else 4000, row['publisher'], faker_br.date_between(start_date='-100y', end_date='-1y')))

            for ath in eval(row['authors']):
                # 3. Inserir em Autor_Livro
                cursor.execute("""
                    INSERT INTO Autorias (id_autor, id_midia)
                    VALUES (%s, %s);
                """, (ath, id_midia))
    except Exception as e:
        continue
        

conn.commit()
del livros

In [ ]:
author_df.loc[df['nome'] == 'Yingxu Wang']['id_autor'].values[0]


In [ ]:
revistas = pd.concat([pd.read_csv(path_join(REVISTA_PATH, file), dtype=str, sep=",") for file in os.listdir(REVISTA_PATH)], ignore_index=True)
leng = revistas.shape[0]
for idx, row in revistas.iterrows():
    print(f"Revistas: {idx}/{leng}")
    biblios = random.randint(1, 2)
    try:
        for id_biblioteca in random.sample(biblio_ids, biblios):
            cursor.execute("""
                INSERT INTO Midia (tipo_midia, condicao, id_biblioteca)
                VALUES (%s, %s, %s)
                RETURNING id_midia;
            """, ('revista', random.choice(['NOVO', 'SEMINOVO', 'DESGASTADO', 'DANIFICADO', 'ARQUIVADO']), id_biblioteca))

            id_midia = cursor.fetchone()[0]


            if not row['ISSN'] or row['ISSN'] == 'nan': 
                row['ISSN'] = faker_en.isbn13()
            
            
            row['data_lancamento'] = faker_br.date_between(start_date='-60y', end_date='today').strftime('%Y-%m-%d')


            # 2. Inserir em Livros (com o mesmo id)
            cursor.execute("""
                INSERT INTO Revistas (id_revista, titulo, ISSN, periodicidade, editora, data_publicacao)
                VALUES (%s, %s, %s, %s, %s, %s);
            """, (id_midia, row['titulo'], row['ISSN'].replace('-', ''), row['periodicidade'], row['editora'], row['data_publicacao']))

            for ath in row['autores'].split('|')[:2]:
                id_autor = int(author_df.loc[author_df['nome'].str.strip() == ath.strip()]['id_autor'].values[0])

                cursor.execute("""
                    INSERT INTO Autorias (id_autor, id_midia)
                    VALUES (%s, %s);
                """, (id_autor, id_midia))
    except Exception as e:
        print(e)
        continue
        

conn.commit()
del revistas

In [ ]:
# artigos = pd.concat([pd.read_csv(path_join(ARTIGO_PATH, file), dtype=str, sep=",") for file in os.listdir(ARTIGO_PATH)], ignore_index=True)
# pd.read_csv('artigos/treated/arXiv_scientific_dataset.csv').isnull().sum()  

In [ ]:
# artigos = pd.concat([pd.read_csv(path_join(ARTIGO_PATH, file), dtype=str, sep=",") for file in os.listdir(ARTIGO_PATH)], ignore_index=True)
# leng = artigos.shape[0]
# for idx, row in artigos.iterrows():
#     print(f"Artigos: {idx}/{leng}")
#     biblios = random.randint(1, 5)
#     try:
#         for id_biblioteca in random.sample(biblio_ids, biblios):
#             cursor.execute("""
#                 INSERT INTO Midia (tipo_midia, condicao, id_biblioteca)
#                 VALUES (%s, %s, %s)
#                 RETURNING id_midia;
#             """, ('artigo', random.choice(['NOVO', 'SEMINOVO', 'DESGASTADO', 'DANIFICADO', 'ARQUIVADO']), id_biblioteca))

#             id_midia = cursor.fetchone()[0]

#             if not row['DOI']: 
#                 row['DOI'] = faker_en.isbn13()
            
#             if not row['data_publicacao'] or row['data_publicacao'] == 'nan': 
#                 row['data_publicacao'] = faker_br.date_between(start_date='-60y', end_date='today').strftime('%Y-%m-%d')

#             # 2. Inserir em Livros (com o mesmo id)
#             cursor.execute("""
#                 INSERT INTO artigos (id_artigo, titulo, DOI, publicadora, data_publicacao)
#                 VALUES (%s, %s, %s, %s, %s);
#             """, (id_midia, row['titulo'], row['DOI'].replace('-', '').replace('/', ''), row['publicadora'], row['data_publicacao']))

#             for ath in row['autores'].split('|'):
#                 id_autor = int(author_df.loc[df['nome'] == 'Yingxu Wang']['id_autor'].values[0])

#                 cursor.execute("""
#                     INSERT INTO Autorias (id_autor, id_midia)
#                     VALUES (%s, %s);
#                 """, (id_autor, id_midia))
#     except Exception as e:
#         print(e)
#         continue
        

# conn.commit()
# del artigos

In [ ]:
# dvds = pd.concat([pd.read_csv(path_join(DVD_PATH, file), dtype=str, sep=",") for file in os.listdir(DVD_PATH)], ignore_index=True)

# dvds.isnull().sum()

In [ ]:
conn = psycopg2.connect(
    dbname="onixlibrary",
    user="super_user",
    password="carimboatrasado",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

dvds = pd.concat([pd.read_csv(path_join(DVD_PATH, file), dtype=str, sep=",") for file in os.listdir(DVD_PATH)], ignore_index=True)
dvds['data_lancamento'] = dvds['data_lancamento'].astype(str)
leng = dvds.shape[0]
for idx, row in dvds.iterrows():
    try:
        print(f"DVDs: {idx}/{leng}")
        biblios = random.randint(1, 5)
        for id_biblioteca in random.sample(biblio_ids, biblios):
            cursor.execute("""
                INSERT INTO Midia (tipo_midia, condicao, id_biblioteca)
                VALUES (%s, %s, %s)
                RETURNING id_midia;
            """, ('dvd', random.choice(['NOVO', 'SEMINOVO', 'DESGASTADO', 'DANIFICADO', 'ARQUIVADO']), id_biblioteca))

            id_midia = cursor.fetchone()[0]

            try: 
                duracao = int(row['duracao']) 
                if duracao > 5000:
                    duracao = 120
            except ValueError:
                duracao = 120

            try:
                # print(row['data_lancamento'])
                lancamento = dt.strptime(row['data_lancamento'], "%Y-%m-%d").date()
            except Exception as e:
                lancamento= faker_br.date_between(start_date='-100y', end_date='today').strftime('%Y-%m-%d')


            # 2. Inserir em Livros (com o mesmo id)
            cursor.execute("""
                INSERT INTO dvds (id_dvd, titulo, ISAN, duracao, distribuidora, data_lancamento)
                VALUES (%s, %s, %s, %s, %s, %s);
            """, (id_midia, row['titulo'], str(row['ISAN']).replace('-', ''), duracao , row['distribuidora'], lancamento))
    except Exception as e:
        print(e)
        continue
        
conn.commit()
del dvds

In [ ]:

from datetime import timedelta

# Conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="onixlibrary",
    user="super_user",
    password="carimboatrasado",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Parâmetros de quantidade
NUM_EMPRESTIMOS = 5_954_982
PENALIZACAO_PERCENTUAL = 0.1
NUM_PENALIZACOES = int(NUM_EMPRESTIMOS * PENALIZACAO_PERCENTUAL)


# IDs de mídia e usuários existentes
cursor.execute("SELECT id_midia FROM Midia")
midias = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT id_usuario FROM Usuario")
usuarios = [row[0] for row in cursor.fetchall()]

emprestimos = []
penalizacoes = []

# Geração de empréstimos
for _ in range(NUM_EMPRESTIMOS):
    id_usuario = random.choice(usuarios)
    id_midia = random.choice(midias)

    data_emprestimo = faker_br.date_between(start_date="-20y", end_date="today")
    data_prevista = data_emprestimo + timedelta(days=14)

    devolvido = random.choices([True, False], weights=[90, 10])[0]
    if devolvido:
        atraso = random.randint(0, 10)
        data_devolucao = data_prevista + timedelta(days=atraso)
    else:
        data_devolucao = None


    cursor.execute("""
        INSERT INTO Emprestimo (data_emprestimo, data_devolucao_prevista, data_devolucao, id_midia, id_usuario)
        VALUES (%s, %s, %s, %s, %s);
    """, (data_emprestimo, data_prevista, data_devolucao, id_midia, id_usuario))


# conn.commit()
cursor.execute('SELECT id_emprestimo, id_usuario, data_devolucao FROM Emprestimo')

# Geração de penalizações (10% aleatórios dos empréstimos)
for id_emprestimo, id_usuario, devolucao in random.sample(cursor.fetchall(), NUM_PENALIZACOES):
    try:
        if id_usuario not in usuarios:
            print(f"Usuário {id_usuario} não encontrado.")
            continue
        descricao = faker_br.sentence(nb_words=50)
        fim_penalizacao = faker_br.date_between(start_date=devolucao if devolucao else "-1y", end_date="+90d")

        cursor.execute("""
            INSERT INTO Penalizacao (descricao, Final_penalizacao, id_usuario, id_emprestimo)
            VALUES (%s, %s, %s, %s);
        """, (descricao, fim_penalizacao, id_usuario, id_emprestimo))
    except Exception as e:
        print(f"Erro ao inserir penalização: {e}")
        continue


# Commit e fechamento
conn.commit()
cursor.close()
conn.close()
